In [11]:
# 引入包
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import time
import bs4
import json
from datetime import datetime

In [12]:
# 随机产生user_agent，避免被禁
def get_agent_pc():
    user_agent_pc = [
        # 谷歌
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.html.2171.71 Safari/537.36',
        'Mozilla/5.0.html (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.html.1271.64 Safari/537.11',
        'Mozilla/5.0.html (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.html.648.133 Safari/534.16',
        # 火狐
        'Mozilla/5.0.html (Windows NT 6.1; WOW64; rv:34.0.html) Gecko/20100101 Firefox/34.0.html',
        'Mozilla/5.0.html (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
        # opera
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.html.2171.95 Safari/537.36 OPR/26.0.html.1656.60',
        # qq浏览器
        'Mozilla/5.0.html (compatible; MSIE 9.0.html; Windows NT 6.1; WOW64; Trident/5.0.html; SLCC2; .NET CLR 2.0.html.50727; .NET CLR 3.5.30729; .NET CLR 3.0.html.30729; Media Center PC 6.0.html; .NET4.0C; .NET4.0E; QQBrowser/7.0.html.3698.400)',
        # 搜狗浏览器
        'Mozilla/5.0.html (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.html.963.84 Safari/535.11 SE 2.X MetaSr 1.0.html',
        # 360浏览器
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.html.1599.101 Safari/537.36',
        'Mozilla/5.0.html (Windows NT 6.1; WOW64; Trident/7.0.html; rv:11.0.html) like Gecko',
        # uc浏览器
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.html.2125.122 UBrowser/4.0.html.3214.0.html Safari/537.36',
    ]
    return random.choice(user_agent_pc)

In [13]:
# 请求的cookie
cookies = {
    'qgqp_b_id': 'd140d591678662b7c1b9fb74c1ad64ab',
    'HAList': 'ty-1-000001-%u4E0A%u8BC1%u6307%u6570%2Cty-1-601318-%u4E2D%u56FD%u5E73%u5B89%2Cty-1-000002-%uFF21%u80A1%u6307%u6570%2Cty-1-600519-%u8D35%u5DDE%u8305%u53F0',
    'st_si': '52449960593274',
    'st_asi': 'delete',
    'st_pvi': '39951595300255',
    'st_sp': '2023-03-06%2009%3A49%3A34',
    'st_inirUrl': 'https%3A%2F%2Fcn.bing.com%2F',
    'st_sn': '11',
    'st_psi': '20230406125155117-113200313003-9059891318',
}
# 请求头
headers = {
    'Accept': '*/*',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Referer': 'http://quote.eastmoney.com/',
    'User-Agent': get_agent_pc(),
}
# 参数设置
jq = re.sub('\D','','1.12.4'+str(random.random()))
new_ts = time.time()
tm = int(new_ts*1000)
jQuery = 'jQuery{}_{}'.format(jq,tm)
params={'cb':jQuery,
        "pn":1,
        "pz":50,
        "po":1,
        "np":1,
        "ut":"bd1d9ddb04089700cf9c27f6f7426281",
        "fltt":2,
        "invt":2,
        "wbp2u":"|0|0|0|web",
        "fid":"",
        "fs":"b:MK0010",
        "fields":"f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f26,f22,f11,f62,f128,f136,f115,f152",
        '_':'{}'.format(tm)
        }

In [14]:
# 设置存储的dataframe
stock_df = pd.DataFrame(columns=["时间",'代码','名称',"最新价","涨跌幅",'涨跌额',
                                        "成交量","成交额","昨收",'今开','最高','最低'])

In [15]:
# 爬取数据的链接
url = 'http://43.push2.eastmoney.com/api/qt/clist/get'
# 发送请求
response = requests.get(
    url=url,
    cookies=cookies,
    headers=headers,
    params=params
)

# 返回请求状态
print(response.status_code)
if response.status_code==200:
    # 如果成功
    response.encoding = "utf-8"
    text = response.text
    #解析返回的内容
    html = text.lstrip(jQuery+'(')
    html = html.rstrip(');')
    #转换为json数据
    data_json = json.loads(html)
    data = data_json['data']
    # 获取我们要的数据
    rows = data['diff']
    print(data)
    # ["时间",'代码','名称',"最新价","涨跌幅",'涨跌额',"成交量","成交额","昨收",'今开','最高','最低']
    for row in rows:
        # 将dict结构转化为dataframe
        code = row['f12']
        name = row['f14']
        new = row['f2']
        dt = datetime.fromtimestamp(new_ts)
        new_time = dt.strftime("%Y-%m-%d %H:%M:%S")
        price = row['f4']
        price_diff = row['f3']
        turnover = row['f5']
        turnover_num = row['f6']
        yest =row['f18']
        today = row['f17']
        hight = row['f15']
        low = row['f16']
        stock_df.loc[len(stock_df)]=[new_time,code,name,new,price,price_diff,turnover,turnover_num,yest,today,hight,low]


200
{'total': 29, 'diff': [{'f1': 2, 'f2': 3324.4, 'f3': 0.36, 'f4': 11.77, 'f5': 173930714, 'f6': 274070915541.8, 'f7': 0.63, 'f8': 0.41, 'f9': '-', 'f10': 1.05, 'f11': 0.03, 'f12': '000001', 'f13': 1, 'f14': '上证指数', 'f15': 3328.85, 'f16': 3308.06, 'f17': 3312.48, 'f18': 3312.63, 'f20': 50460414435538, 'f21': 43714793124320, 'f22': 0.02, 'f23': '-', 'f24': 5.36, 'f25': 7.61, 'f26': '-', 'f62': -4826368000.0, 'f115': '-', 'f128': '-', 'f140': '-', 'f141': '-', 'f136': '-', 'f152': 2}, {'f1': 2, 'f2': 11964.89, 'f3': 0.83, 'f4': 98.58, 'f5': 269534531, 'f6': 391169088301.2502, 'f7': 1.01, 'f8': 1.21, 'f9': '-', 'f10': 1.2, 'f11': 0.0, 'f12': '399001', 'f13': 0, 'f14': '深证成指', 'f15': 11978.79, 'f16': 11858.5, 'f17': 11867.87, 'f18': 11866.31, 'f20': 35658653461493, 'f21': 29227308069351, 'f22': 0.01, 'f23': '-', 'f24': 5.58, 'f25': 8.61, 'f26': '-', 'f62': -6392669952.0, 'f115': '-', 'f128': '-', 'f140': '-', 'f141': '-', 'f136': '-', 'f152': 2}, {'f1': 2, 'f2': 948.56, 'f3': 0.89, 'f4':

In [16]:
# 输出前5行
pd.set_option('display.width', 100)
stock_df.head(10)

,时间,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,昨收,今开,最高,最低
0,2023-04-07 12:50:33,000001,上证指数,3324.40,11.77,0.36,173930714,2.740709e+11,3312.63,3312.48,3328.85,3308.06
1,2023-04-07 12:50:33,399001,深证成指,11964.89,98.58,0.83,269534531,3.911691e+11,11866.31,11867.87,11978.79,11858.50
2,2023-04-07 12:50:33,899050,北证50,948.56,8.40,0.89,251700,2.348020e+08,940.16,941.06,949.85,940.89
3,2023-04-07 12:50:33,000300,沪深300,4122.19,25.55,0.62,80040459,1.830919e+11,4096.64,4096.90,4131.31,4092.18
4,2023-04-07 12:50:33,399005,中小100,7918.51,61.05,0.78,20820508,5.284503e+10,7857.46,7859.55,7935.54,7852.42
5,2023-04-07 12:50:33,399006,创业板指,2449.15,22.58,0.93,99031956,1.830733e+11,2426.57,2421.75,2454.04,2421.06
6,2023-04-07 12:50:33,000010,上证180,8653.22,42.64,0.50,50993520,9.769439e+10,8610.58,8612.95,8674.59,8603.39
7,2023-04-07 12:50:33,000016,上证50,2684.44,17.99,0.67,15076254,3.596794e+10,2666.45,2667.14,2692.51,2663.61
8,2023-04-07 12:50:33,000009,上证380,6044.29,27.77,0.46,30496244,5.299374e+10,6016.52,6016.97,6048.61,6008.09
9,2023-04-07 12:50:33,000132,上证100,6536.91,37.99,0.58,4911034,1.108326e+10,6498.92,6505.51,6540.45,6495.29


In [17]:
# 保存成文件
stock_df.to_csv("沪深重要指数.csv",index=False)

In [18]:
cookies = {
    'qgqp_b_id': 'd140d591678662b7c1b9fb74c1ad64ab',
    'st_si': '59910575272429',
    'st_asi': 'delete',
    'em-quote-version': 'topspeed',
    'HAList': 'ty-1-600519-%u8D35%u5DDE%u8305%u53F0%2Ca-sh-600519-%u8D35%u5DDE%u8305%u53F0%2Cty-1-000001-%u4E0A%u8BC1%u6307%u6570%2Cty-1-601318-%u4E2D%u56FD%u5E73%u5B89%2Cty-1-000002-%uFF21%u80A1%u6307%u6570',
    'st_pvi': '39951595300255',
    'st_sp': '2023-03-06%2009%3A49%3A34',
    'st_inirUrl': 'https%3A%2F%2Fcn.bing.com%2F',
    'st_sn': '7',
    'st_psi': '20230407123244880-113200301201-1939509406',
}

headers = {
    'Accept': '*/*',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Referer': 'http://quote.eastmoney.com/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}
url = "http://push2his.eastmoney.com/api/qt/stock/trends2/get"
new_ts = time.time()
tm = int(new_ts*1000)
cb = "jsonp"+str(tm)
params = {
    "fields1":"f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13",
    "fields2":"f51,f52,f53,f54,f55,f56,f57,f58",
    "secid":"1.600519",
    "ndays":1,
    "iscr":0,
    "iscca":0,
    "ut":"fa5fd1943c7b386f172d6893dbfba10b",
    "cb":cb

}
response = requests.get(
    url=url,
    params =params,
    cookies=cookies,
    headers=headers,
    verify=False,
)
his_df = pd.DataFrame(columns=["时间",'最新价','均价',"成交额"])
# 返回请求状态
print(response.status_code)
if response.status_code==200:
    # 如果成功
    response.encoding = "utf-8"
    text = response.text
    #解析返回的内容
    html = text.lstrip(cb+'(')
    html = html.rstrip(');')
    #转换为json数据
    data_json = json.loads(html)
    data = data_json['data']
    # 获取我们要的数据
    rows = data['trends']
    # print(rows)
    # # ["时间",'代码','名称',"最新价","涨跌幅",'涨跌额',"成交量","成交额","昨收",'今开','最高','最低']
    for row in rows:
        li = row.split(",")
        his_df.loc[len(his_df)]=[li[0],li[2],li[-1],li[-2]]


200


In [19]:
# 输出前5行
pd.set_option('display.width', 100)
his_df.head(10)

,时间,最新价,均价,成交额
0,2023-04-07 09:30,1795.00,1795.000,13462500.00
1,2023-04-07 09:31,1791.00,1793.647,88596008.00
2,2023-04-07 09:32,1796.00,1793.605,14884515.00
3,2023-04-07 09:33,1796.34,1793.973,20303059.00
4,2023-04-07 09:34,1797.00,1794.333,18688662.00
5,2023-04-07 09:35,1798.00,1794.584,14019519.00
6,2023-04-07 09:36,1798.00,1794.846,12588722.00
7,2023-04-07 09:37,1797.13,1795.147,18520700.00
8,2023-04-07 09:38,1798.92,1795.442,19782901.00
9,2023-04-07 09:39,1799.15,1795.662,14631165.00


In [20]:
# 保存成文件
his_df.to_csv("贵州茅台历史数据.csv",index=False)